In [5]:
import pandas as pd
from datetime import datetime

In [96]:
us_BreitbartNews = pd.read_json('data/us-BreitbartNews.json', lines=True)
us_cnn = pd.read_json('data/us-cnn.json', lines=True)
us_huffpost = pd.read_json('data/us-huffpost.json', lines=True)
us_latimes = pd.read_json('data/us-latimes.json', lines=True)
us_msnbc = pd.read_json('data/us-msnbc.json', lines=True)
us_nytimes = pd.read_json('data/us-nytimes.json', lines=True)
us_usatoday = pd.read_json('data/us-usatoday.json', lines=True)
us_washingtonpost = pd.read_json('data/us-washingtonpost.json', lines=True)
us_wsj = pd.read_json('data/us-wsj.json', lines=True)

In [97]:
usData = us_BreitbartNews.append(us_cnn).append(us_huffpost).append(us_latimes).append(us_msnbc).append(us_nytimes).append(us_usatoday).append(us_washingtonpost).append(us_wsj)

C:\Users\Erkan Er\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [99]:
usData["created_at"] = [d.date() for d in usData["created_at"]]

In [100]:
usData["created_at"].unique()

array([datetime.date(2020, 4, 17), datetime.date(2020, 4, 16),
       datetime.date(2020, 4, 15), datetime.date(2020, 4, 18),
       datetime.date(2020, 4, 19), datetime.date(2020, 4, 20),
       datetime.date(2020, 4, 21), datetime.date(2020, 4, 22)],
      dtype=object)

In [101]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if ((token.orth_.isspace()) | (token.orth_ == 'another') | (token.orth_ == 'continue')):
            continue
        elif token.like_url:
            lda_tokens.append('URL')        
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [102]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to C:\Users\Erkan
[nltk_data]     Er\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [103]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\Erkan
[nltk_data]     Er\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [104]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [105]:
text_data = []

for line in usData['full_text']:
    tokens = prepare_text_for_lda(line)

    text_data.append(tokens)

In [106]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [107]:
import gensim
NUM_TOPICS = 15
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model15.gensim')
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.037*"coronavirus" + 0.032*"business" + 0.029*"small" + 0.019*"pandemic" + 0.016*"program"')
(1, '0.058*"trump" + 0.039*"president" + 0.037*"coronavirus" + 0.033*"reopen" + 0.030*"state"')
(2, '0.046*"coronavirus" + 0.019*"economic" + 0.019*"pandemic" + 0.019*"donâ€" + 0.019*"american"')
(3, '0.022*"world" + 0.018*"coronavirus" + 0.016*"health" + 0.012*"pandemic" + 0.011*"things"')
(4, '0.018*"coronavirus" + 0.016*"college" + 0.014*"california" + 0.013*"hotel" + 0.011*"break"')
(5, '0.018*"biden" + 0.015*"earth" + 0.013*"trump" + 0.013*"years" + 0.013*"change"')
(6, '0.035*"coronavirus" + 0.020*"quarantine" + 0.020*"every" + 0.020*"around" + 0.017*"chinese"')
(7, '0.032*"billion" + 0.020*"street" + 0.019*"coronavirus" + 0.018*"american" + 0.017*"journal"')
(8, '0.017*"school" + 0.016*"student" + 0.014*"explain" + 0.014*"employee" + 0.013*"company"')
(9, '0.030*"coronavirus" + 0.027*"order" + 0.014*"protest" + 0.014*"lockdown" + 0.012*"protester"')
(10, '0.030*"market" + 0.024*"re

In [108]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model15.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\Erkan Er\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


# Day 1:  2020, 4, 15

In [109]:
usData_firstDate = usData[usData['created_at'] == datetime(2020, 4, 15).date()]

In [110]:
first_text_data = []

for line in usData_firstDate['full_text']:
    tokens = prepare_text_for_lda(line)    
    
    first_text_data.append(tokens)

In [111]:
from gensim import corpora
dictionary = corpora.Dictionary(first_text_data)
corpus = [dictionary.doc2bow(text) for text in first_text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [112]:
import gensim
NUM_TOPICS = 15
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model15.gensim')
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.030*"still" + 0.015*"want" + 0.015*"america" + 0.015*"week" + 0.015*"safely"')
(1, '0.019*"america" + 0.019*"obsession" + 0.019*"weeknight" + 0.019*"incredibly" + 0.019*"pasta"')
(2, '0.013*"party" + 0.013*"obama" + 0.013*"establish" + 0.013*"favor" + 0.013*"bidenâ€"')
(3, '0.040*"coronavirus" + 0.032*"trump" + 0.024*"state" + 0.024*"pandemic" + 0.016*"administration"')
(4, '0.019*"coronavirus" + 0.019*"social" + 0.019*"distance" + 0.010*"times" + 0.010*"force"')
(5, '0.012*"today" + 0.012*"facing" + 0.012*"country" + 0.012*"patient" + 0.012*"journal"')
(6, '0.029*"coronavirus" + 0.012*"federal" + 0.012*"volunteer" + 0.012*"college" + 0.012*"delivery"')
(7, '0.035*"story" + 0.024*"adjourn" + 0.024*"congress" + 0.024*"threaten" + 0.024*"trump"')
(8, '0.017*"pandemic" + 0.017*"write" + 0.017*"child" + 0.017*"@nytparenting" + 0.017*"doctor"')
(9, '0.027*"coronavirus" + 0.018*"people" + 0.018*"covid-19" + 0.018*"american" + 0.010*"lose"')
(10, '0.028*"funding" + 0.028*"trump" + 0.02

In [113]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model15.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\Erkan Er\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


# Day 8:  2020, 4, 22

In [114]:
usData_eightDate = usData[usData['created_at'] == datetime(2020, 4, 22).date()]

In [115]:
eight_text_data = []

for line in usData_eightDate['full_text']:
    tokens = prepare_text_for_lda(line)    
    
    eight_text_data.append(tokens)

In [116]:
from gensim import corpora
dictionary = corpora.Dictionary(eight_text_data)
corpus = [dictionary.doc2bow(text) for text in eight_text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [117]:
import gensim
NUM_TOPICS = 15
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model15.gensim')
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.025*"coronavirus" + 0.022*"pandemic" + 0.009*"month" + 0.008*"trump" + 0.007*"world"')
(1, '0.038*"coronavirus" + 0.008*"county" + 0.007*"pandemic" + 0.007*"infection" + 0.006*"death"')
(2, '0.031*"coronavirus" + 0.007*"worker" + 0.007*"pandemic" + 0.007*"large" + 0.006*"line"')
(3, '0.018*"state" + 0.009*"people" + 0.008*"coronavirus" + 0.008*"order" + 0.008*"south"')
(4, '0.028*"coronavirus" + 0.011*"earth" + 0.010*"business" + 0.009*"health" + 0.008*"reopen"')
(5, '0.019*"trump" + 0.018*"coronavirus" + 0.012*"stimulus" + 0.012*"business" + 0.012*"president"')
(6, '0.017*"coronavirus" + 0.017*"people" + 0.012*"testing" + 0.011*"hospital" + 0.008*"covid-19"')
(7, '0.022*"coronavirus" + 0.014*"people" + 0.008*"trump" + 0.007*"discus" + 0.006*"first"')
(8, '0.016*"coronavirus" + 0.008*"covid-19" + 0.008*"trump" + 0.008*"program" + 0.008*"different"')
(9, '0.025*"coronavirus" + 0.022*"death" + 0.020*"california" + 0.014*"trump" + 0.012*"state"')
(10, '0.011*"mayor" + 0.008*"busine

In [118]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model15.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\Erkan Er\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
